<a href="https://colab.research.google.com/github/lmlucania/Titanic_Pytorch/blob/main/Titanic_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%ls

drive/  sample_data/


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("drive/My Drive/タイタニック/train.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

欠損値を置き換える

In [6]:
df.fillna(df.Age.median(), inplace=True)
df.fillna(df.Embarked.mode(), inplace=True)

In [7]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [8]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
dataset = df[['Survived','Pclass','Sex','Age','Fare']]

In [10]:
dataset_dummies=pd.get_dummies(dataset)

In [11]:
dataset_dummies.head()

,Survived,Pclass,Age,Fare,Sex_female,Sex_male
0,0,3,22.0,7.2500,0,1
1,1,1,38.0,71.2833,1,0
2,1,3,26.0,7.9250,1,0
3,1,1,35.0,53.1000,1,0
4,0,3,35.0,8.0500,0,1


In [12]:
X = dataset_dummies.iloc[:, 1:]
y = dataset_dummies.iloc[:, 0]

In [13]:
X.head()

,Pclass,Age,Fare,Sex_female,Sex_male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [15]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [17]:
X_train = torch.Tensor(X_train.values)
X_test = torch.Tensor(X_test.values)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values) 

In [18]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 10)
        self.fc2 = nn.Linear(10, 20)
        self.fc3 = nn.Linear(20, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return F.log_softmax(x, dim=1)

In [19]:
model = Net()

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.02)

In [21]:
num_epochs = 2000
train_losses = []
train_accs = []
test_losses = []
test_accs = []

for epoch in range(num_epochs):
    model.train()
    data, target = Variable(X_train), Variable(y_train)
    optimizer.zero_grad()
    output = model(data) 
    train_loss = criterion(output, target)
    train_loss.backward()
    train_losses.append(train_loss.data.item())
    optimizer.step()
    prediction = output.data.max(1)[1]
    train_acc = prediction.eq(target.data).sum().numpy() / len(X_train)
    train_accs.append(train_acc)
    #test loop
    model.eval()
    data, target = Variable(X_test), Variable(y_test)
    output = model(data)
    test_loss = criterion(output, target)
    test_losses.append(test_loss.data.item())
    prediction = output.data.max(1)[1]
    test_acc = prediction.eq(target.data).sum().numpy() / len(X_train)
    test_accs.append(test_acc)

    if epoch % 100 == 0:
        print('Epoch: {}\ttrain_loss: {:.3f}\ttrain_acc: {:.3f}\ttest_loss: {:.3f}\ttest_acc: {:.3f}'.format(epoch, train_loss.data.item(), train_acc, test_loss.data.item(), test_acc))



Epoch: 0	train_loss: 0.748	train_acc: 0.575	test_loss: 0.743	test_acc: 0.210
Epoch: 100	train_loss: 0.594	train_acc: 0.692	test_loss: 0.590	test_acc: 0.231
Epoch: 200	train_loss: 0.582	train_acc: 0.693	test_loss: 0.582	test_acc: 0.234
Epoch: 300	train_loss: 0.568	train_acc: 0.689	test_loss: 0.574	test_acc: 0.234
Epoch: 400	train_loss: 0.567	train_acc: 0.693	test_loss: 0.572	test_acc: 0.231
Epoch: 500	train_loss: 0.560	train_acc: 0.699	test_loss: 0.568	test_acc: 0.235
Epoch: 600	train_loss: 0.560	train_acc: 0.690	test_loss: 0.563	test_acc: 0.234
Epoch: 700	train_loss: 0.562	train_acc: 0.692	test_loss: 0.571	test_acc: 0.237
Epoch: 800	train_loss: 0.558	train_acc: 0.686	test_loss: 0.563	test_acc: 0.237
Epoch: 900	train_loss: 0.560	train_acc: 0.683	test_loss: 0.566	test_acc: 0.237
Epoch: 1000	train_loss: 0.575	train_acc: 0.701	test_loss: 0.559	test_acc: 0.238
Epoch: 1100	train_loss: 0.531	train_acc: 0.746	test_loss: 0.549	test_acc: 0.246
Epoch: 1200	train_loss: 0.507	train_acc: 0.743	test_